In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [53]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import math
import gc

In [58]:
ROOT = "/Users/kkalyan/github/airQo/"
train = pd.read_csv(ROOT+ "data/Train.csv")
test = pd.read_csv(ROOT+ "data/Test.csv")

In [59]:
def replace_nan(x):
    if x==" ":
        return np.nan
    else :
        return float(x)
features=["temp","precip","rel_humidity","wind_dir","wind_spd","atmos_press"]
for feature in features : 
    train[feature]=train[feature].apply(lambda x: [ replace_nan(X) for X in x.replace("nan"," ").split(",")])
    test[feature]=test[feature].apply(lambda x: [ replace_nan(X)  for X in x.replace("nan"," ").split(",")])   

In [61]:
def aggregate_features(x,col_name):
    x["max_"+col_name]=x[col_name].apply(np.max)
    x["min_"+col_name]=x[col_name].apply(np.min)
    x["mean_"+col_name]=x[col_name].apply(np.mean)
    x["std_"+col_name]=x[col_name].apply(np.std)
    x["var_"+col_name]=x[col_name].apply(np.var)
    x["median_"+col_name]=x[col_name].apply(np.median)
    x["ptp_"+col_name]=x[col_name].apply(np.ptp)
    return x  
def remove_nan_values(x):
    return [e for e in x if not math.isnan(e)]

In [62]:
data=pd.concat([train,test],sort=False).reset_index(drop=True)
data.columns.tolist()

['ID',
 'location',
 'temp',
 'precip',
 'rel_humidity',
 'wind_dir',
 'wind_spd',
 'atmos_press',
 'target']

In [63]:
for x in range(121):
    data["newtemp"+ str(x)] = data.temp.str[x]
    data["newprecip"+ str(x)] = data.precip.str[x]
    data["newrel_humidity"+ str(x)] = data.rel_humidity.str[x]
    data["newwind_dir"+ str(x)] = data.wind_dir.str[x]
    data["windspeed"+ str(x)] = data.wind_spd.str[x]
    data["atmospherepressure"+ str(x)] = data.atmos_press.str[x]

In [67]:
data.columns

Index(['ID', 'location', 'temp', 'precip', 'rel_humidity', 'wind_dir',
       'wind_spd', 'atmos_press', 'target', 'newtemp0',
       ...
       'newrel_humidity119', 'newwind_dir119', 'windspeed119',
       'atmospherepressure119', 'newtemp120', 'newprecip120',
       'newrel_humidity120', 'newwind_dir120', 'windspeed120',
       'atmospherepressure120'],
      dtype='object', length=735)

In [68]:
for col_name in tqdm(features):
    data[col_name]=data[col_name].apply(remove_nan_values)

100%|██████████| 6/6 [00:02<00:00,  2.38it/s]


In [69]:
for col_name in tqdm(features):
    data=aggregate_features(data,col_name)

100%|██████████| 6/6 [00:17<00:00,  2.90s/it]


In [70]:
data.drop(features,1,inplace=True)

In [72]:
data.columns

Index(['ID', 'location', 'target', 'newtemp0', 'newprecip0',
       'newrel_humidity0', 'newwind_dir0', 'windspeed0', 'atmospherepressure0',
       'newtemp1',
       ...
       'var_wind_spd', 'median_wind_spd', 'ptp_wind_spd', 'max_atmos_press',
       'min_atmos_press', 'mean_atmos_press', 'std_atmos_press',
       'var_atmos_press', 'median_atmos_press', 'ptp_atmos_press'],
      dtype='object', length=771)

In [73]:
train=data[data.target.notnull()].reset_index(drop=True)
test=data[data.target.isna()].reset_index(drop=True)
del data  
gc.collect()

88

In [76]:
ROOT="/Users/kkalyan/github/airQo/data/"

In [77]:
train.to_csv(ROOT+"train-proc.csv",index  = False)
test.to_csv(ROOT+"test-proc.csv",index  = False)